In [0]:
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size,modificationTime
dbfs:/FileStore/tables/Book-1.csv,Book-1.csv,83,1715692755000
dbfs:/FileStore/tables/Book.csv,Book.csv,81,1715692059000
dbfs:/FileStore/tables/Book1-1.xlsx,Book1-1.xlsx,10465,1714557239000
dbfs:/FileStore/tables/Book1.xlsx,Book1.xlsx,10465,1714556901000
dbfs:/FileStore/tables/azure_data_engineering_questions-1.txt,azure_data_engineering_questions-1.txt,574,1715704624000
dbfs:/FileStore/tables/azure_data_engineering_questions.txt,azure_data_engineering_questions.txt,574,1715254051000
dbfs:/FileStore/tables/json.txt,json.txt,191,1715613058000
dbfs:/FileStore/tables/json_csv.xlsx,json_csv.xlsx,8824,1715613854000
dbfs:/FileStore/tables/mart_in_json.json,mart_in_json.json,818,1717074694000
dbfs:/FileStore/tables/multilinejson.csv,multilinejson.csv,216,1715613980000


In [0]:
df_json=spark.read.option("Multiline", True).json("/FileStore/tables/mart_in_json.json")
df_json.printSchema()

root
 |-- Appliance: struct (nullable = true)
 |    |-- item: string (nullable = true)
 |    |-- rate: double (nullable = true)
 |-- Area: string (nullable = true)
 |-- Month: long (nullable = true)
 |-- Stationary: struct (nullable = true)
 |    |-- item: string (nullable = true)
 |    |-- rate: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Vegetable: struct (nullable = true)
 |    |-- item: string (nullable = true)
 |    |-- rate: double (nullable = true)
 |-- fruits: struct (nullable = true)
 |    |-- item: string (nullable = true)
 |    |-- rate: double (nullable = true)
 |-- year: long (nullable = true)



In [0]:
#Solution add a new struct column 

#import struct function
from pyspark.sql.functions import *

#Get Existing Struct Schema of fruits from dataframe
s_fields = df_json.schema["fruits"].dataType.names
print(s_fields)

df2 = (
    df_json.withColumn(
        "fruits",
        struct(
            *(
                [col('fruits')[c].alias(c) for c in s_fields]
                + [lit(10).alias('discount')]
            )
        )
    )
)



['item', 'rate']


In [0]:
s_vegetable=df_json.schema['Vegetable'].dataType.names
print(s_vegetable)


['item', 'rate']


In [0]:
df_veg=df2.withColumn("Vegetable",struct(*[col('Vegetable')[c] for c in s_vegetable] + [lit(10).alias('discount')]

))
display(df_veg)

Appliance,Area,Month,Stationary,Type,Vegetable,fruits,year
"List(Mixer, 113.2)",CHN004,5,"List(Pencil, 3.1)",Retail Super Market,"List(tomato, 5.25, 10)","List(apple, 10.25, 10)",2022
"List(Mixer, 103.9)",DLN004,5,"List(Pencil, 4.1)",Retail Super Market,"List(tomato, 9.25, 10)","List(apple, 12.15, 10)",2022
"List(Mixer, 117.7)",MUM011,5,"List(Pencil, 3.7)",Retail Super Market,"List(tomato, 5.1, 10)","List(apple, 9.5, 10)",2022


In [0]:
#get the existing schema
s_fields = df1.schema["Vegetable"].dataType.names

#Remove the column from the list 
s_fields.remove("rate")

#define the column with removed schema
df3=df2.withColumn("Vegetable",struct(*([col('Vegetable')[c].alias(c) for c in s_fields])))

#New dataframe
df3.printSchema()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2448447228025846>:2
      1 #get the existing schema
----> 2 s_fields = df1.schema["Vegetable"].dataType.names
      4 #Remove the column from the list 
      5 s_fields.remove("rate")

NameError: name 'df1' is not defined

In [0]:
# cast columns inside struct field
#Get the schema
s_fields = df1.schema["Stationary"].dataType.names
s_fields.remove("rate")


#cast Rate column as string
df4=(
  df3.withColumn("Stationary",
                 struct(*([col('Stationary')[c].alias(c) for c in s_fields] + 
                          [col('Stationary')['rate'].cast('string').alias('rate')]))
                )
)

#print Schema
df4.printSchema()

In [0]:
display(df_veg)

Appliance,Area,Month,Stationary,Type,Vegetable,fruits,year
"List(Mixer, 113.2)",CHN004,5,"List(Pencil, 3.1)",Retail Super Market,"List(tomato, 5.25, 10)","List(apple, 10.25, 10)",2022
"List(Mixer, 103.9)",DLN004,5,"List(Pencil, 4.1)",Retail Super Market,"List(tomato, 9.25, 10)","List(apple, 12.15, 10)",2022
"List(Mixer, 117.7)",MUM011,5,"List(Pencil, 3.7)",Retail Super Market,"List(tomato, 5.1, 10)","List(apple, 9.5, 10)",2022


In [0]:
#to cast string
df_collist=df_veg.schema['Vegetable'].dataType.names
df_cast=df_veg.withColumn('Vegetable',struct(*[col('Vegetable')[c].cast('String').alias(c) for c in df_collist]))
# to cast all column to string
df_vegetable=df_cast.select('Vegetable')

# for struct column we cannot use json_tuple to flatten the data so we can use struct function itself
df_flatten = df_vegetable.select(
    *[col(f'Vegetable.{c}').alias(f'Vegetable_{c}') for c in df_collist]
)

df_flatten.show(truncate=0)




+--------------+--------------+------------------+
|Vegetable_item|Vegetable_rate|Vegetable_discount|
+--------------+--------------+------------------+
|tomato        |5.25          |10                |
|tomato        |9.25          |10                |
|tomato        |5.1           |10                |
+--------------+--------------+------------------+



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2448447228025850>:1
----> 1 df_explode=df_vegetable.select('Vegetable',explode('Vegetable'))
      2 df_json=df_vegetable.select('Vegetable',to_json(col('Vegetable')).alias('Vegetable'))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions and returns a new :class:`DataFrame`.
   2980 
 

In [0]:
#df_flatten1=df_vegetable.select('Vegetable',json_tuple(col('Vegetable'),'item','rate','discount'))